In [8]:
%load_ext autoreload
%autoreload 2

import os
ROOT_DIR = os.getcwd()[:os.getcwd().rfind('NVcenter')]+ 'NVcenter'
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from NVcenter import *
import numpy as np

In [3]:
# spin_baths = save_spin_baths('C13_bath_configs', DATA_DIR, 'C13', 0.02e-2, 2e-9, 4.2e-9, 10, 10)
bath_configs = list(load_spin_baths('C13_bath_configs', DATA_DIR).values())

In [4]:
register_config = [('NV', (0, 0, 0), 0, {}), ('C13', (4.722331100730915e-10, 0.0, 1.030637866442101e-10), 0, {})]
bath_config = SpinBath('C13', 0.02e-2, 2e-9, 4.2e-9).config
approx_level = 'no_bath'

hamiltonian = Hamiltonian(register_config, bath_config, approx_level)
print( hamiltonian.matrices[0] )

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[-7.92420875e+04  0.00000000e+00  3.88908730e+04  1.78196033e+05]
 [ 0.00000000e+00  7.92420875e+04  5.37401154e+04 -3.88908730e+04]
 [ 3.88908730e+04  5.37401154e+04  2.45326996e+09 -5.50000000e+04]
 [ 1.78196033e+05 -3.88908730e+04 -5.50000000e+04  2.45327645e+09]]


In [5]:
# targets
H_gate = q.tensor( q.qeye(2), 1/np.sqrt(2) * (q.sigmax() + q.sigmaz()) )
CNOT_gate = q.tensor( q.fock_dm(2,0), q.qeye(2) ) + q.tensor( q.fock_dm(2,1), q.sigmax() )
init_state = q.tensor( q.fock_dm(2,0), q.fock_dm(2,0) )
H_state = H_gate * init_state * H_gate.dag()
CNOT_state = CNOT_gate * init_state * CNOT_gate.dag()

# with the correct factor 2pi: each vector gets multiplied by 1/np.cbrt(2*np.pi)=0.54
# Suter_C13_1 = (4.722331100730915e-10, 0.0, 1.030637866442101e-10)
# Suter_C13_2 = (3.6747967115510627e-10, 0.0, 1.4217244863175118e-10)
# Suter_C13_3 = (4.1283485801972827e-10, 0.0, 8.964851198130749e-11)
# Suter_C13_4 = (3.6919326714940366e-10, 0.0, 8.849893347694991e-11)

print( H_Suter() )

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[-79000.      0.      0.      0.]
 [     0.  79000.      0.      0.]
 [     0.      0.  -3000. -55000.]
 [     0.      0. -55000.   3000.]]


In [9]:
C13_pos = (4.722331100730915e-10, 0.0, 1.030637866442101e-10)
calc_hadamard_pulse_seq(C13_pos)

NameError: name 'calc_hadamard_pulse_seq' is not defined

In [25]:
class Pulse(Hamiltonian):
    def __init__(self, pulse_seq, register_config, bath_config, approx_level, target, 
                 dynamical_decoupling=False, old_state=None, mode='state_preparation', instant_pulses=False):
        """
        Notes:
            - Note: the before and after the pulses should be a free time evolution such that 
        the free time list has one more entry than the pulse time list.
            - Available modes: 'state_preparation', 'unitary_gate'  
        """
        
        super().__init__(register_config, bath_config, approx_level)
        self.pulse_seq = pulse_seq
        self.target = target
        self.dynamical_decoupling = dynamical_decoupling
        self.mode = mode
        self.instant_pulses = instant_pulses

        # starting state (can be a quantum state or a quantum gate)
        if old_state is None and self.mode == 'state_preparation':
            self.old_states = self.calc_system_states(self.register_init_state)

        elif old_state is None and self.mode == 'unitary_gate':
            self.old_states = self.calc_system_states( q.tensor([q.qeye(2) for _ in range(self.register_num_spins)]) )

        else:
            self.old_states = self.calc_system_states(old_state)

        # Gates
        self.omega_L = DEFAULTS['omega_L']
        self.XGate = self.calc_U_rot(np.pi, 0, theta=np.pi/2) # -1j X
        self.HGate = self.calc_U_rot(np.pi, 0, theta=np.pi/4) # -1j H

        # Pulse sequence
        self.num_pulses = (len(self.pulse_seq)-1)//3
        self.free_time_list = self.pulse_seq[:self.num_pulses+1]
        if not self.instant_pulses:
            self.pulse_time_list = self.pulse_seq[self.num_pulses+1:2*self.num_pulses+1]
        else: 
            self.alpha_list = self.pulse_seq[self.num_pulses+1:2*self.num_pulses+1]
        self.phi_list = self.pulse_seq[2*self.num_pulses+1:]

        self.cumulative_time_list = self.calc_cumulative_time_list()
        self.total_time = self.cumulative_time_list[-1]

        self.pulse_matrices = self.calc_pulse_matrices(self.total_time)
        self.new_states = self.calc_new_states(self.total_time)
        self.fidelities = [calc_fidelity(dm, self.target) for dm in self.new_states]

    def calc_cumulative_time_list(self):
        """ Calculates the cumulative time (free time evolution and pulse time).  """
        
        if self.instant_pulses:
            num_time_steps = len(self.free_time_list)
            return [sum(self.free_time_list[:i+1]) for i in range(num_time_steps)]
            
        num_time_steps = len(self.free_time_list) + len(self.pulse_time_list)
        full_time_list = [0] * num_time_steps
        for i in range(len(self.pulse_time_list)):
            full_time_list[2*i] = self.free_time_list[i]
            full_time_list[2*i+1] = self.pulse_time_list[i]
        full_time_list[-1] = self.free_time_list[-1]
        cumulative_time_list = [sum(full_time_list[:i+1]) for i in range(num_time_steps)]
        return cumulative_time_list

    # ------------------------------------------------

    def calc_H_rot(self, omega_L, phi, theta=np.pi/2):
        """ Returns a Hamiltonian that rotates the first register spin (NV center) with the Lamor 
        frequency around an axis determined by spherical angles. """
        
        n = np.array([spherical_to_cartesian(1, phi, theta)])
        H_rot = omega_L * np.sum( n * get_spin_matrices(1/2)[1:] ) # factor 2 to get Pauli matrices
        H_rot = adjust_space_dim(self.system_num_spins, H_rot, 0)  
        return H_rot.to(data_type="CSR")

    def calc_U_rot(self, alpha, phi, theta=np.pi/2):
        """ Returns the unitary gate that rotates the first register spin (NV center) by an 
        angle alpha around an axis determined by spherical angles. """
        
        t = 1 # arbitrary value bacuse it cancels
        omega_L = alpha / t
        H_rot = self.calc_H_rot(omega_L, phi, theta=theta)
        return (-1j * t * H_rot).expm()

    def calc_U_time(self, eigv, eigs, time):
        """ Returns the unitary gate for the time evolution given the eigenenergies and eigenstates of an Hamiltonian. """
        
        U_time = eigs @ np.diag(np.exp(-1j * eigv * time)) @ eigs.conj().T
        U_time = q.Qobj(U_time, dims=[[2]*self.system_num_spins, [2] * self.system_num_spins])
        return U_time.to(data_type="CSR")

    # -------------------------------------------

    def get_reduced_pulse_seq(self, t):
        """ Returns the pulse sequence for an arbitrary time. """
        
        if t >= self.total_time:
            free_time_list = self.free_time_list
            free_time_list[-1] += t-self.total_time
            if not self.instant_pulses:
                return free_time_list, self.pulse_time_list, self.phi_list
            else: 
                return free_time_list, self.alpha_list, self.phi_list
            
        indices = [i+1 for i, value in enumerate(self.cumulative_time_list) if value <= t]
        finished_time_steps = indices[-1] if indices else 0  
        
        if finished_time_steps == 0:
            left_time = t
            return [t], [], []
            
        left_time = t - self.cumulative_time_list[finished_time_steps-1]

        print(finished_time_steps)
        finished_free_time_steps = finished_time_steps//2 + finished_time_steps%2
        finished_pulse_time_steps = finished_time_steps//2
        
        phi_list = self.phi_list[:finished_pulse_time_steps]
        pulse_time_list = self.pulse_time_list[:finished_pulse_time_steps]
        free_time_list = self.free_time_list[:finished_free_time_steps]
        
        if left_time != 0 and finished_time_steps%2==0:
            free_time_list.append(left_time)
        if left_time != 0 and finished_time_steps%2!=0:
            pulse_time_list.append(left_time)
            phi_list.append( self.phi_list[finished_pulse_time_steps] )
            free_time_list.append(0) # because the pulse sequence has to end with a free evolution
            
        return free_time_list, pulse_time_list, phi_list

    # ---------------------------------------------------

    def calc_pulse_matrix(self, pulse_seq, free_matrix):
        """ Calculates the pulse matrix for a given pulse sequence and Hamiltonian. """
        
        free_matrix *= 2*np.pi # very important!!!
        omega_L = 2*np.pi*self.omega_L # Lamor frequency as angular frequency
        eigv_free, eigs_free = np.linalg.eigh( free_matrix.full())

        if not instant_pulses:
            free_time_list, pulse_time_list, phi_list = pulse_seq
        else:
            free_time_list, alpha_list, phi_list = pulse_seq
        
        U_list = []
        for i in range(self.num_pulses):

            # free time evolution
            if not self.dynamical_decoupling: 
                U_time = self.calc_U_time(eigv_free, eigs_free, free_time_list[i])
            else:
                U_half_time = self.calc_U_time(eigv_free, eigs_free, free_time_list[i]/2)
                U_time = U_half_time * self.XGate * U_half_time
            U_list.append(U_time)

            # rotation 
            if not self.instant_pulses:
                rot_matrix = self.calc_H_rot(omega_L, phi_list[i])
                eigv_rot, eigs_rot = np.linalg.eigh( (free_matrix + rot_matrix).full() )
                U_rot = self.calc_U_time(eigv_rot, eigs_rot, pulse_time_list[i])
            else:
                U_rot = self.calc_U_rot(alpha_list[i], phi_list[i])
            
            U_list.append(U_rot)

        # free evolution after the last pulse
        U_list.append(self.calc_U_time(eigv_free, eigs_free, free_time_list[-1]))

        # construct pulse_matrix from list of unitary gates
        pulse_matrix = self.spin_ops[0][0] # identity
        for U in U_list[::-1]: # see eq. (14) in Dominik's paper
            pulse_matrix *= U
        return pulse_matrix

    def calc_pulse_matrices(self, t):
        """ Calculates the pulse matrices for each system at a given time t. """
        
        pulse_seq = self.get_reduced_pulse_seq(t)
        print(pulse_seq)
        pulse_matrices = []
        for matrix in self.matrices:
            pulse_matrix = self.calc_pulse_matrix(pulse_seq, matrix) 
            pulse_matrices.append( pulse_matrix )
        return pulse_matrices

    def calc_new_states(self, t):
        """ Calculates the new states for the register at a given time t. """
        
        new_states = []
        if t == self.total_time:
            pulse_matrices = self.pulse_matrices
        else:
            pulse_matrices = self.calc_pulse_matrices(t)
        
        for pulse_matrix, old_state in zip(pulse_matrices, self.old_states):
            
            if self.mode == 'state_preparation':
                new_state = pulse_matrix * old_state * pulse_matrix.dag()
            if self.mode == 'unitary_gate':
                new_state = pulse_matrix * old_state

            # reduce from system to register space by tracing out
            reduced_new_state = q.ptrace(new_state, np.arange(self.register_num_spins))
            new_states.append(reduced_new_state)
        return new_states

In [26]:
register_config = [('NV', (0, 0, 0), 0, {}), ('C13', , 0, {})]
bath_config = SpinBath('C13', 0.02e-2, 2e-9, 4.2e-9).config
approx_level = 'no_bath'
target = H_gate

pulse_seq = [0.74e-6, 0.22e-6, 0.43e-6, 0.89e-6, 0.23e-6, 1.26e-6, 1.50e-6, 3*np.pi/2, 3*np.pi/2, np.pi/2] # Suter Hadamard 
pulse = Pulse(pulse_seq, register_config, bath_config, approx_level, target, old_state=None, mode='unitary_gate' )

t_list = np.linspace(0, pulse.total_time, 20)
gate_list = [pulse.calc_new_states(t) for t in t_list]

fig, ax = plt.subplots()
ax.plot(t_list, [gate[0,0] for gate in gate_list])
ax.plot(t_list, [gate[0,1] for gate in gate_list])
ax.plot(t_list, [gate[0,2] for gate in gate_list])
ax.plot(t_list, [gate[0,3] for gate in gate_list])

([7.4e-07, 2.2e-07, 4.3e-07, 8.9e-07], [2.3e-07, 1.26e-06, 1.5e-06], [4.71238898038469, 4.71238898038469, 1.5707963267948966])


NameError: name 'instant_pulses' is not defined

In [19]:
register_config = [('NV', (0, 0, 0), 0, {}), ('C13', (4.722331100730915e-10, 0.0, 1.030637866442101e-10), 0, {})]
bath_config = SpinBath('C13', 0.02e-2, 2e-9, 4.2e-9).config
approx_level = 'gCCE0'

pulse_seq = [0] # Suter Hadamard 
target = H_gate

pulse = Pulse(pulse_seq, register_config, bath_config, approx_level, target, old_state=None, mode='state_preparation' )

T1, T2 = calc_hadamard_analytical()

pi_pulse = pulse.calc_U_rot(np.pi, 0)
H = 2*np.pi * pulse.matrices[0].full()
eigv, eigs = np.linalg.eigh( H )
free_evo1 = pulse.calc_U_time(eigv, eigs, T1)
free_evo2 = pulse.calc_U_time(eigv, eigs, T2)

old_state = pulse.init_states[0]
pulse_matrix = free_evo2 * pi_pulse * free_evo1 * pi_pulse
new_state = pulse_matrix * old_state * pulse_matrix.dag()
print( new_state[1, 1].real )

0.4999587814818642


In [78]:
register_config = [('NV', (0, 0, 0), 0, {}), ('C13', (0.87e-9, 0, 0.19e-9), 0, {})]
bath_config = SpinBath('C13', 0.02e-2, 2e-9, 4.2e-9).config
approx_level = 'no_bath'

# abundancy = 0.005e-2 # Suter, 20us T2 time
# pulse_seq = [0e-6, 4.06e-6, 1.57e-6, 1.51e-6, 1.00, 3.58, 1.68, 0.69, 1.97, 0.50] # Dominik Hadamard
pulse_seq = [0.74e-6, 0.22e-6, 0.43e-6, 0.89e-6, 0.23e-6, 1.26e-6, 1.50e-6, 3*np.pi/2, 3*np.pi/2, np.pi/2] # Suter Hadamard 
# pulse_seq = [3.78e-6, 2.11e-6, 2.15e-6, 0.63e-6, 1.88e-6, 3.96e-6, 1.9e-6, 0, np.pi/5, np.pi/2] # Suter CNOT

target = H_gate

pulse = Pulse(pulse_seq, register_config, bath_config, approx_level, target, old_state=None, mode='unitary_gate' )
pulse.new_states

[Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=False
 Qobj data =
 [[-3.12019800e-01-0.89688678j -1.89857145e-01-0.01660386j
    1.08852956e-01-0.15287263j  1.10112782e-01+0.12071657j]
  [-1.87893442e-01-0.04013895j  6.55742167e-01-0.70036547j
   -2.25924539e-02+0.06491034j  8.77583515e-02+0.17363767j]
  [-8.56397006e-02-0.07426137j -1.46465514e-02-0.20202737j
   -1.08657780e-01-0.30731433j -8.51775543e-01-0.33815873j]
  [ 1.95736026e-01-0.1008026j   2.40841455e-04-0.04625322j
   -8.65762234e-01-0.32289993j  2.79485067e-01-0.13216016j]]]

In [113]:
class Environment:
    def __init__(self, register_config, bath_config, approx, target, dynamical_decoupling=False):

        self.register_config = register_config
        self.bath_config = bath_config
        self.approx = approx
        self.target = target
        self.dynamical_decoupling = dynamical_decoupling

        self.hamiltonian = Hamiltonian(self.register_config, self.bath_config, 'no_bath')
        self.register_init_state = self.hamiltonian.register_init_state
        
        # reset emvironment
        self.state = self.register_init_state
        self.fidelity = calc_fidelity(self.state, self.target)

    def reset(self):
        self.state = self.init_state
        self.fidelity = calc_fidelity(self.state, self.target)

    def step(self, pulse_seq):
        if self.approx == 'gCCE':
            self.state = self.get_new_states_gCCE(pulse_seq, self.state, 2)
        else: 
            self.state = self.get_new_states(pulse_seq, self.state, self.approx)[0]
            
        self.fidelity = calc_fidelity(self.state, self.target)
        return self.state, self.fidelity
        

    def get_new_states(self, pulse_seq, old_state, approx_level):
        pulse = Pulse(pulse_seq, 
                      self.register_config, 
                      self.bath_config, 
                      approx_level, 
                      self.target, 
                      old_state=old_state, 
                      dynamical_decoupling=self.dynamical_decoupling)

        if approx_level == 'gCCE2':
            return pulse.new_states, pulse.idx_gCCE2
        return pulse.new_states
    
    def get_new_states_gCCE(self, pulse_seq, old_states, gCCE_order):
        gCCE0_dm = self.get_new_states(pulse_seq, old_states, 'gCCE0')[0]
        gCCE_dm = gCCE0_dm
        
        if gCCE_order == 0:
            return gCCE_dm
            
        gCCE1_dms = self.get_new_states(pulse_seq, old_states, 'gCCE1')
        gCCE_dm_correction = np.prod( [gCCE1_dm.full() / gCCE0_dm.full() for gCCE1_dm in gCCE1_dms], axis=0 )
        gCCE_dm_correction = q.Qobj(gCCE_dm, dims=[[2, 2], [2, 2]])
        gCCE_dm *= gCCE_dm_correction
        
        if gCCE_order == 1:
            return gCCE_dm
            
        gCCE2_dms, idx_gCCE2 = self.get_new_states(pulse_seq, old_states, 'gCCE2') 
        gCCE_dm_correction = np.prod( [(gCCE2_dm.full() * gCCE0_dm.full()) / (gCCE1_dms[i].full() * gCCE1_dms[j].full()) for gCCE2_dm, (i,j) in zip(gCCE2_dms, idx_gCCE2)], axis=0 )
        gCCE_dm_correction = q.Qobj(gCCE_dm, dims=[[2, 2], [2, 2]])
        gCCE_dm *= gCCE_dm_correction
        
        if gCCE_order == 2:
            return gCCE_dm
            
        else:
            return NotImplementedError

In [115]:
register_config = [('NV', (0, 0, 0), 0, {}), ('C13', (0.87e-9, 0, 0.19e-9), 0, {})]
bath_config = SpinBath('C13', 0.02e-2, 2e-9, 2.5e-9).config
approx = 'full_bath'

pulse_seq = [0e-6, 4.06e-6, 1.57e-6, 1.51e-6, 1.00, 3.58, 1.68, 0.69, 1.97, 0.50]
target = 1/np.sqrt(2) * (q.tensor(q.fock_dm(2,0), q.fock_dm(2,0)) + q.tensor(q.fock_dm(2,1), q.fock_dm(2,1)) )

env = Environment(register_config, bath_config, approx, target, dynamical_decoupling=False)
env.step(pulse_seq)[1]

0.4355367060849788